# Retrieval API DEMO

## Code Initialization
Dependencies and environment initialization.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `.env.example` file as template.

In [1]:
import os
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
USER_EMAIL = 'retrieval.api@mycompany.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [3]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token created for user {USERNAME.split('@')[0].split('-')[0]}"))
else:
    display(Markdown(f"**Authentication Failed**: Cannot obtain the Bearer token for the user {USERNAME.split('@')[0].split('-')[0]}"))

**Authentication Successful**: Bearer token created for user 9ZZZ159100

This creates the headers to be used in the Retrieval API requests.

Additionally, `user_email_hash` is calculated as `user_id`. This must be a unique and constant value per user, so the API statistics keep track of the right number of users. The recommendation is calculating a MD5 hash of the user email. This value is always the same and do not expose any PII.

In the case of `work_id`, this value must be calculated differently for each API request. For convenience, it can be taken from the environment's transaction if it exists, or calculated randomly before calling the API. In this case it is calculated randomly before the API request. See the payload section below.

In [4]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

user_email_hash = u.md5_hash(USER_EMAIL)

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [5]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla"

### Search Filters
* **Language**: Factiva language code(s), composed of two characters. Please see [Taxonomy - Languages](9_taxonomy_languages.ipynb)
* **Organization**: Factiva company/orgnization code(s). Please see [Taxonomy - Organizations](9_taxonomy_organizations.ipynb)
* **News Subjects**: Factiva company/orgnization code(s). Please see [Taxonomy - News Subjects](9_taxonomy_news_subjects.ipynb)
* **Industries**: Factiva Industries. Please see [Taxonomy - Industries](9_taxonomy_industries.ipynb)
* **Sources**: Factiva Sources (publishers). Please see [Taxonomy - Sources](9_taxonomy_sources.ipynb)
* **Regions**: Factiva Regions. Please see [Taxonomy - Regions](9_taxonomy_regions.ipynb)


### API Payload
Assemble the Retrieval API payload

In [6]:
work_id = u.generate_random_guid()

frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 10,    # max 50
      "query": {
        "value": frapi_prompt,
        "search_filters": [
          { "scope": "Language", "value": "en or es or it" },
          { "scope": "Organization", "value": "TESLMI" },
          { "scope": "NewsSubject", "value": "C151" },
          { "scope": "Industry", "value": "IAUT" },
          { "scope": "Source", "value": "DJDN or J or WSJO or B" },
          { "scope": "Region", "value": "USA" }
        ],
        "date": {
          "custom": {
            "from": "2025-04-01",
            "to": "2025-05-31"
          }
        }
      },
      "metrics_data": {
        "user_id": user_email_hash,
        "work_id": work_id,
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}

## Send Retrieval API Request and Receive Chunks Back

In [7]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print & Save Chunks

In [8]:
chunks_list = chunks_resp.json()['data']

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

###                            Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com

**Dow Jones Newswires** - 2025-05-18 - DJDN000020250518el5i00017 - Lang: en

Al Root Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com Al Root Investors are looking for evidence that brand "challenges" ca...

---

###                            Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com

...

---

###                            Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role -- WSJ

**Dow Jones Institutional News** - 2025-04-22 - DJDN000020250422el4m003sa - Lang: en

Tesla's net income slid 71% in the first quarter, as the company struggled to overcome competitive pressure overseas and a reputational hit from Chief...

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example the retrieved articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is **Google Gemini 2.5 Flash Preview 05-20**. Response generations took a few seconds for a context between 5K to 8K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Gemini Structured Prompt

In [9]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format:
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [10]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250518el5i00017",
            "source_name": "Dow Jones Newswires",
            "headline": "Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com",
            "publication_date": "2025-05-18",
            "content": "Al Root Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com Al Root Investors are looking for evidence that brand \"challenges\" ca..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58004jd",
            "source_name": "Dow Jones Institutional News",
            "headline": "Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com",
            "publication_date": "2025-05-08

### Gemini Request/Response

In [12]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"**LLM Model:** {GCP_MODEL}"))
display(Markdown(f"**Prompt:** {frapi_prompt} \n --------------------"))
display(Markdown(response))

**LLM Model:** gemini-2.5-flash-preview-05-20

**Prompt:** Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla 
 --------------------

During April and May 2025, the U.S. administration made several tariff-related decisions. In early May, U.S. and Chinese officials announced a 90-day pause on steep import tariffs, which had previously been over 100%, following talks in Switzerland [4]. Additionally, President Donald Trump's trade deal with the U.K. included smaller tariffs on vehicle imports from Britain and lower U.K. levies on U.S. cars [2]. Prior to May 6, President Trump also altered auto tariff policies, providing manufacturers with some relief on imported parts [7].

These decisions had a mixed impact on Tesla:

*   **Positive Impacts:**
    *   The 90-day pause on U.S.-China tariffs was seen as an improvement in trade relations and contributed to a rise in Tesla's stock [4]. This pause was anticipated to help, as Chinese consumers had been avoiding American products due to escalating trade tensions [4].
    *   The U.S.-U.K. trade deal, which included lower tariffs, was considered a "boon for all auto makers" [2].
    *   The relief on imported parts tariffs was also viewed as a positive development [7].

*   **Ongoing Challenges/Negative Impacts (from existing tariff regime):**
    *   Despite some relief, tariffs continued to stress supply chains and add to automakers' costs [8, 9].
    *   Tesla, while assembling all cars sold domestically in the U.S., still faced tariffs on some imported parts [3, 7, 10]. CEO Elon Musk noted that tariffs were particularly tough given the company's low operating margins [3, 7].
    *   The broader "trade war" with China continued to influence Chinese car buyers to favor local models, contributing to significant year-over-year sales declines for Tesla in China, including a 15% drop in the five weeks ending May 4 and a 56% drop in early May [2, 4, 6].
    *   The administration's tariff regime was cited as a factor in Tesla's 71% net income slide in the first quarter of 2025, stressing supply chains and adding costs [8, 9, 10]. The trade fight and changing political sentiment were also noted as potential factors weighing on demand for Tesla vehicles [8, 9].

### Cited Articles

*   [1] [Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com - Dow Jones Newswires - 2025-05-18](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250518el5i00017)
*   [2] [Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com - Dow Jones Institutional News - 2025-05-08](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58004jd)
*   [3] [Tesla Stock Falls Amid Tariffs Turmoil. What Ford Earnings Mean for Auto Makers. -- Barrons.com - Dow Jones Institutional News - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56000ps)
*   [4] [Tesla Stock Drops on Fall in Chinese Sales, Trump's Plan to Kill EV Credit -- Barrons.com - Dow Jones Institutional News - 2025-05-13](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250513el5d002pj)
*   [5] [Tesla's China Sales Stink. Why the Stock Is Rising Anyway. -- Barrons.com - Dow Jones Newswires - 2025-05-20](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250520el5k002n6)
*   [6] [Tesla Stock Is Rising. There's Little Evidence of a Post-DOGE Sales Recovery. -- Barrons.com - Dow Jones Institutional News - 2025-05-08](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58002y4)
*   [7] [Tesla Stock Falls as Sales in Germany Tumble. It's About More Than Musk. -- Barrons.com - Dow Jones Institutional News - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56001q4)
*   [8] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role -- Update - Dow Jones Newswires - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250422el4m003rc)
*   [9] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role; CEO says he will spend significantly less time on DOGE starting in May; electric-vehicle maker's net income falls 71% - The Wall Street Journal Online - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.WSJO000020250422el4m0053d)
*   [10] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role -- WSJ - Dow Jones Institutional News - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250422el4m003sa)